# Démo minimaliste RAG (Retrieval Augmented Generation)

Ce notebook montre comment combiner la recherche documentaire (retrieval) et la génération de texte (LLM) pour répondre à une question utilisateur de façon augmentée.

In [ ]:
# Installation des dépendances
!pip install sentence-transformers faiss-cpu transformers

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline

# 1. Mini-dataset
documents = [
    "La RAG combine la recherche documentaire et la génération de texte.",
    "LangChain et LlamaIndex sont des frameworks populaires pour la RAG.",
    "L'indexation vectorielle permet de retrouver rapidement des passages pertinents.",
    "La génération augmentée améliore la précision des réponses IA."
]

# 2. Embedding
embedder = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = embedder.encode(documents)

# 3. Indexation (FAISS)
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)

# 4. Question utilisateur
question = "Quels frameworks utiliser pour faire de la RAG ?"
q_embedding = embedder.encode([question])

# 5. Recherche du passage le plus pertinent
D, I = index.search(q_embedding, k=1)
retrieved = documents[I[0][0]]
print("Passage retrouvé :", retrieved)

# 6. Génération de réponse augmentée (pipeline HuggingFace)
gen_pipe = pipeline("text-generation", model="gpt2")
prompt = f"Question : {question}\nContexte : {retrieved}\nRéponse :"
result = gen_pipe(prompt, max_new_tokens=40)[0]['generated_text']
print(result)